# Detecting Insults in Social Networks

![](https://pbs.twimg.com/media/CkEyfjKUUAURpd9.jpg)

In [ ]:
import re, string

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC

##  Load raw data

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.findall(r'\b[a-z]+\b', text)
    return ' '.join(text)

In [ ]:
training_data = pd.read_csv('train.csv')
training_data.head(7)

In [ ]:
training_data['cleaned_comment'] = training_data['Comment'].map(clean_text)

In [ ]:
training_data.head(7)

## Make some features

In [ ]:
count_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,3), stop_words='english', max_features=50000)
count_vectorizer.fit(training_data['cleaned_comment'])

In [ ]:
X = count_vectorizer.transform(training_data['cleaned_comment'])
y = training_data['Insult']

## Cross-validate

### Split data

In [ ]:
mask = [bool(np.random.binomial(1, .75)) for _ in range(X.shape[0])]

In [ ]:
mask

In [ ]:
X[np.array(mask)]

In [ ]:
sum(mask) 

In [ ]:
y[mask].shape[0]

In [ ]:
y[~mask].shape

In [ ]:
def split_data(X, y, p=.75):
    mask = np.array([bool(np.random.binomial(1, p)) for _ in range(X.shape[0])])
    
    X_train = X[mask]
    y_train = y[mask]
    X_validation = X[~mask]
    y_validation = y[~mask]
    
    return X_train, y_train, X_validation, y_validation

In [ ]:
X_train, y_train, X_validation, y_validation = split_data(X, y)

### Fit a model on training data

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

### Validate model on validation data

In [ ]:
baseline_predictions = np.zeros(predictions.shape[0])

In [ ]:
from sklearn.metrics import accuracy_score


predictions = model.predict(X_validation)
validation_score = accuracy_score(y_validation, predictions)

print('Validation Score:', validation_score)

In [ ]:
baseline_validation_score = accuracy_score(y_validation, baseline_predictions)

print('Validation Score:', baseline_validation_score)

## Remember, everything is a hyper-parameter..

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
class PredictionPipeline:
    
    def __init__(self, ngram_range, vectorizer_class, model_class, training_data):
        self.ngram_range=ngram_range
        self.vectorizer_class=vectorizer_class
        self.model_class=model_class
        self.training_data=training_data
        self.vectorizer = None
        self.X = None
        self.y = None
        self.model = None
        self.validation_score = None
        
    def run(self):
        self._fit_vectorizer()
        self._featurize_text()
        self._split_train_and_validation_sets()
        self._fit_model_on_training_data()
        self._validate_model_on_validation_set()
        
        print(
            """
            Vectorizer Class: {vectorizer_class}\n\
            N-gram Range: {ngram_range}\n\
            Model Class: {model_class}\n\
            Validation Score: {validation_score}
            """.format(

            vectorizer_class=repr(self.vectorizer_class.__name__), 
            ngram_range=self.ngram_range, 
            model_class=repr(self.model_class.__name__), 
            validation_score=round(self.validation_score, 4)

            )
        )

    def _fit_vectorizer(self):
        self.vectorizer = vectorizer_class(analyzer='word', ngram_range=ngram_range, 
                                     stop_words='english', max_features=50000)
        self.vectorizer.fit(self.training_data['cleaned_comment'])
    
    def _featurize_text(self):
        self.X = self.vectorizer.transform(self.training_data['cleaned_comment'])
        self.y = self.training_data['Insult']

    def _split_train_and_validation_sets(self):
        self.X_train, self.y_train, self.X_validation, self.y_validation = split_data(
            self.X, self.y)

    def _fit_model_on_training_data(self):
        self.model = self.model_class()
        self.model.fit(self.X_train, self.y_train)

    def _validate_model_on_validation_set(self):
        predictions = self.model.predict(self.X_validation)
        self.validation_score = accuracy_score(self.y_validation, predictions)      

In [ ]:
results = {}

for ngram_range in [(1, 1), (1, 2), (1, 3), (1, 4)]:
    for vectorizer_class in [CountVectorizer, TfidfVectorizer]:
        for model_class in [LogisticRegression, LinearSVC, RandomForestClassifier]:
            
            # run prediction pipeline
            prediction_pipeline = PredictionPipeline(
                ngram_range=ngram_range,
                vectorizer_class=vectorizer_class,
                model_class=model_class,
                training_data=training_data
            )
            
            prediction_pipeline.run()
            
            # add hyper-parameters to `results` dictionary
            results[str(prediction_pipeline.validation_score)] = {
                    'vectorizer_class': prediction_pipeline.vectorizer_class,
                    'ngram_range': prediction_pipeline.ngram_range,
                    'model_class': prediction_pipeline.model_class
            }

In [ ]:
top_3_scores = sorted(results.keys(), reverse=True)[:3]

for score in top_3_scores:
    print('Score: {score}\nParameters: {parameters}\n'.format(
        score=score, parameters=results[score]))

## Train final model

In [ ]:
top_score_key = top_3_scores[0]

In [ ]:
vectorizer_class = results[top_score_key]['vectorizer_class']
ngram_range = results[top_score_key]['ngram_range']
model_class = results[top_score_key]['model_class']

# fit vectorizer
vectorizer = vectorizer_class(analyzer='word', ngram_range=ngram_range, stop_words='english', max_features=50000)
vectorizer.fit(training_data['cleaned_comment'])

# transform text
X = vectorizer.transform(training_data['cleaned_comment'])
y = training_data['Insult']

# fit model on training data
model = model_class()
model.fit(X, y)

## Run it live

In [ ]:
while True:
    input_string = input('Please enter a string: ')
    input_string = clean_text(input_string)
    x_test = vectorizer.transform([input_string])
    
    prediction = model.predict(x_test)[0]
    print('Insult?: {}'.format( bool(prediction)))